### Définition 1
Soient $\phi:V	\to V$ une transformation linéaire d'un espace vectoriel $V$ et $\lambda \in \mathbb{R}$ une valeur propre de $\phi.$ Alors l'espace propre de $\phi$ associé à $\lambda$ est le sous-ensemble de $V$ défini par $E_{\lambda}=\{v\in V: \phi(v)=\lambda v\}$.

De mani&egrave;re similaire, si $\lambda\in \mathbb{R}$ est une valeur propre de la matrice $A\in M_{n \times n}(\mathbb{R}),$ alors l'espace propre de $A$ associé à $\lambda$ est le sous-ensemble de $M_{n \times 1}(\mathbb{R})$ défini par $E_{\lambda}=\{X\in M_{n \times 1}(\mathbb{R}) : AX=\lambda X\}$.

### Proposition 2
Le sous-ensemble $E_\lambda$ est un sous-espace vectoriel.

In [ ]:
import numpy as np
import plotly
import plotly.graph_objects as go
import sympy as sp
import sys, os 
sys.path.append('../Librairie')
import AL_Fct as al
from IPython.utils import io
from IPython.display import display, Latex
from Ch8_lib import *

In [ ]:
#A_sp = sp.Matrix([[0, -4, -1], [0, -8, 8], [0, -4, -4]])
#display(A_sp.rref())
#A_np = np.array([[0, -4, -1], [0, -8, 8], [0, -4, -4]])
#b = np.zeros(3)
#display(al.echelonMat('ER', A))
#A = np.array([[4, -1, 6],[2, 1, 6], [2, -1, 8]])

#A = np.random.randint(-5,5, (3,3))
#A = np.eye(3)
A = sp.Matrix([[3, -4, -1], [0, -5, 8], [0, -1, -4]])
A_sp = sp.Matrix(A)

eig = A_sp.eigenvects()
eig_list = np.array(eig[0][2]).astype(np.float64)

for sp_mat in eig_list:
    sp_mat = np.array(sp_mat)
    
basis, basic_idx, free_idx = eigen_basis_sp(A_sp, float(eig[0][0]), eig_list, disp=True)



In [ ]:
A = np.array([[1, 1, 1],[1, 1, 1], [1, 1, 2]])

A_sp = sp.Matrix(A)

plot3x3_eigspace(A_sp, plot_vector=False)


In [ ]:
A = sp.Matrix([[4, -1, 6],[2, 1, 6], [2, -1, 8]])



In [ ]:
A = sp.Matrix([[1, 1], [-1, 3]])

plot2x2_eigspace(A)